<a href="https://colab.research.google.com/github/CattoYT/ValorantCommsBot/blob/rewrite/Valorant_Bot_LLM_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:

!pip install diskcache flask_ngrok2 pyngrok

# DO NOT SET THIS TO EXTRA INDEX URL, YOU WILL MESS STUFF UP BADLY WIKTH PERFORAMNCE
!pip install -U llama-cpp-python --index-url https://abetlen.github.io/llama-cpp-python/whl/cu122


# a little code is taken from a previous project
from llama_cpp import Llama
class llamaLLM:
    def __init__(self, systemprompt, model="bartowski/Meta-Llama-3-8B-Instruct-GGUF", filename="Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"):


        self.messages = [
            # Sample data fed because I couldn't think of a way to get it to say the right shit
          {"role": "system", "content": systemprompt},
          {"role": "user", "content": "(party) josh: clove hit 70"},
          {"role": "assistant", "content": "[HEALTHINDICATOR] Clove -70"},
          {"role": "user", "content": "(party) josh: brim -140"},
          {"role": "assistant", "content": "[HEALTHINDICATOR] Brimstone -140"},
        ]

        self.llm = Llama.from_pretrained(
            repo_id=model,
            filename=filename,
            verbose=False,
            n_gpu_layers=-1,
            n_ctx=4096
        )
    def addToConversation(self, content, role):
        self.messages.append(
              {
                  "role": role,
                  "content": content
              }
          )
    def generateConversationResponse(self, message, person="user"):
        self.addToConversation(message, person) # it can also be system
        response = self.llm.create_chat_completion(self.messages)["choices"][0]['message']['content'] # forgive me
        self.addToConversation(response, "assistant")
        return response # no genuinely what the fuck is this

llm = llamaLLM("""
You are a gamer playing Valorant, and are talking in the in-game chat. Your teammates may callout damage to certain agents like 'jett 40' or 'sage -120'.
If this sent, you must respond in the format of "[HEALTHINDICATOR] {Agent} {Damange}", where agent, (Any valorant agent) and damage are replaced with the agent and the amount of damage,
but [HEALTHINDICATOR] must be as it is in front. It should not be replaced in any way. The damage should not accumulate. Only output the new damage value and not the total damage

There may be other words such as hit or damaged or any other word like that to reference damage.
Most other information should be removed if it doesn't fit this format
When the message you need to respond to is not a form of callout, in location or damage, you can respond naturally.

The following are valid Valorant Agents:
brimstone phoenix sage sova viper cypher reyna killjoy breach omen jett raze skye yoru astra kayo chamber neon fade harbor gekko deadlock iso clove
There may be abbreviations used for each agent, such as KJ for killjoy or DL for deadlock.
""")

while True:
    print()


Looking in indexes: https://abetlen.github.io/llama-cpp-python/whl/cu122
>> (party) josh: clove hit 70
[HEALTHINDICATOR] Clove -70
>> (party) kaenia: hit kayo 70
[HEALTHINDICATOR] Kayo -70
>> (party) kaenia: kj -90
[HEALTHINDICATOR] Killjoy -90
>> (party) kaenia: brim -110
[HEALTHINDICATOR] Brimstone -110
>> (party) kaenia: Come on guys
What's up?
>> (party) kaenia: Lets win this game already
Let's do this!
>> (party) kaenia: neon -90
[HEALTHINDICATOR] Neon -90
>> (party) kaenia: neon -10
[HEALTHINDICATOR] Neon -10
>> (party) kaenia: astra hit 60
[HEALTHINDICATOR] Astra -60
>> (party) kaenia: astra lit 30
[HEALTHINDICATOR] Astra -30
>> (party) kaenia: kayo -110
[HEALTHINDICATOR] Kayo -110
>> (party) kaenia: kay/o 77
[HEALTHINDICATOR] Kayo 77
>> omen low
Omen's health is low.
>> (party) kaenia: I HIT KAYO 70 WTF
[HEALTHINDICATOR] Kayo -70
>> (party) kaenia: kayo 1 tap
[HEALTHINDICATOR] Kayo -1


KeyboardInterrupt: Interrupted by user

In [ ]:
from flask import Flask, request, jsonify
import threading
import speaker
app = Flask(__name__)
# basic, will improve later

@app.route('/')
def processChat():
    chatmsg = request.args.get("chatmsg")

    return llm.generateConversationResponse(chatmsg)
    pass

def start():
    app.run()